# Przydatne techniki

W tym laboratorium główny nacisk zostanie położony na obsługę wejścia/wyjścia, obsługa popularnych formatów w języku Python (_Excel_/_JSON_/_Parquet_) oraz operacje związane przeglądanie plików i katalogów w systemie plików. Język Python jest niezależny od systemu operacyjnego, który definiuje w jaki sposób fizycznie obsługiwane są pliki. Wiele wysiłku włożono w utworzenie warstwy abstrakcji, która pośredniczy w wywoływaniu funkcji systemowych. Z tego też powodu łączenie katalogu i kolejnych podkatalogów wymaga użycia odpowiedniej funkcji, nie zwykłej konkatenacji łańcuchów znaków zawierających kolejne katalogi. Dodatkowo zostaną przedstawione tematy pokrewne, jak obsługa plikowej bazy danych `sqlite` (transakcyjnej i spełniającej zasady _ACID_) przy użyciu _ORM_ (Object-relational mapping), do czego zostanie użyty moduł `SqlAlchemy` oraz serializacja, deserializacja obiektów przy użyciu modułu `pickle`.

## System plików

Pierwszym przykładem użycia Pythona do obsługi systemu plików będzie rekurencyjne wyświetlenie listy plików. Do tego celu można użyć bardzo przydatnej funkcji `os.walk`, która wyświetla wszystkie pliki i podkatalogi w katalogu podanym w parametrze. Wszystkie nazwy przekazane z funkcji nie zawierają pełnej ścieżki, a jedynie jej relatywną część. W wielu przypadkach potrzebna jest absolutna ścieżka do bieżącego katalogu. Do jej pobrania można użyć wbudowanej funkcji `os.getcwd()` (akronim od _get current working directory_).

In [ ]:
from os import getcwd, walk
from os.path import join

def get_files(folder):
    for folderName, sub_folders, files_names in walk(folder):
        for file_name in files_names:
            print(join(folderName, file_name))
        for sub_folder in sub_folders:
            get_files(join(folder, sub_folder))

get_files(getcwd())

W linii 6 została użyta funkcja `join` modułu `os.path`. Służy do konkatenacji wielu folderów, które składają się na pełną ścieżkę.

Poniższa lista zawiera zestaw funkcji przydatnych do obsługi plików i katalogów na poziomie systemy plików.

| Nazwa funkcji | Opis | Przykład użycia |
|-|-|-|
| `shutil.copy` | Funkcja umożliwia kopiowanie pojedynczych plików. | `shutil.copy(full_path_src, full_path_dest)` |
| `shutil.copytree` | Funkcja umożliwia kopiowanie całych katalogów wraz z zawartością. | `shutil.copytree(path_src, path_dest)` |
| `shutil.move` | Funkcja umożliwia zmianę nazwy pliku oraz przenoszenie pliku do innego katalogu. | `shutil.move(path_src, path_dest)` |
| `shutil.rmtree` | Funkcja umożliwia kasowanie katalogu wraz z zawartością. | `shutil.rmtree(path)` |
| `os.unlink` | Funkcja umożliwia kasowanie pliku. | `shutil.unlink(path)` |
| `os.path.splitext` | Zwraca tuple na, którą składa się nazwa pliku oraz jego rozszerzenie. W przypadku, gdy podany plik zawiera pełną ścieżkę, zostanie ona zwrócona jako część nazwy pliku. | `file_name, ext = os.path.splitext(file_name)` |


Tworzenie bezpiecznych plików tymczasowych jest bardzo przydatne szczególnie w testach jednostkowych. W języku Python dostępny jest moduł `tempfile`, który znacznie ułatwia tworzenie tego typu plików. Funkcje `mkdtemp` i `mktemp` służą kolejno do tworzenia tymczasowego katalogu i pliku.

In [ ]:
from tempfile import mkdtemp
from shutil import rmtree
from os.path import join
temp_dir = mkdtemp()
try:
    print(f'Saving file to: {temp_dir}')
    with open(join(temp_dir, 'file.txt'), 'w+') as f:
        f.write('Test')
        f.flush()
finally:
    rmtree(temp_dir)

print('Success')

W powyższym kodzie to programista jest odpowiedzialny za zwalnianie pliku/katalogu tymczasowego. Moduł `tempfile` umożliwia tworzenie plików, które w momencie opuszczenia bloku kodu, zostaną zwolnione automatycznie.

In [ ]:
from tempfile import NamedTemporaryFile
from os.path import exists

file_name = None
with NamedTemporaryFile('w+') as f:
    f.write('test')
    file_name = f.name
    print(f'Fle {file_name} exists: {exists(file_name)}')

print(f'Fle {file_name} exists: {exists(file_name)}')

Dość istotny jest tutaj fakt, że gdy plik zostanie zamknięty funkcją `f.close()`, zostanie skasowany. W przypadku katalogu można użyć klasy `tempfile.TemporaryDirectory`, której schemat użycia jest taki sam.

## Obsługa formatów plików

### Format `Xlsx`

Moduł `openpyxl` pozwala na obsługę formatu plików `xlsx`. Z tego samego modułu korzysta również bardzo popularny moduł `pandas`, który umożliwia unifikację przetwarzania danych bez względu na format źródłowy. Moduł `openpyxl` nie znajduje się na liście modułów standardowych i należy go doinstalować za pomocą polecenia `pip install openpyxl`.

In [ ]:
from openpyxl import Workbook, load_workbook
from typing import Dict
from tempfile import mktemp
from itertools import product
from os import unlink

def describe(file_name, sheet_name):
    wb = load_workbook(file_name)
    print(f'Sheets: {wb.sheetnames}')
    print(f'Filled rows: {wb[sheet_name].max_row}')
    print(f'Filled columns: {wb[sheet_name].max_column}')
    for cell in product(['A', 'B'], ['1', '2', '3']):
        cell_s = "".join(cell)
        print(f'Cell: {cell_s}: {wb[sheet_name][cell_s].value}')
    # można też użyć
    # for coll_obj in list(wb[sheet_name].columns)[0]:
    #     print(f'Cell: {coll_obj.value}')
    # lub
    # wb[sheet_name].cell(row=1, column=1).value

def create(data: Dict[str, int], file_name, sheet_name):
    wb = Workbook()
    sheet = wb.create_sheet(sheet_name)
    for cell_key, cell_value in data.items():
        sheet[cell_key] = cell_value
    wb.save(file_name)

file_name = mktemp(suffix='.xlsx')

print(f'Using file: {file_name}')

try:
    data = { 'A1': 10, 'A2': 20, 'A3': 30, 'B1': 11, 'B2': 12, 'B3': 13 }
    create(data, file_name, 'test')
    describe(file_name, 'test')

finally:
    unlink(file_name)

Przypisując odpowiedniej komórce formułę w postaci łańcucha znaków tj. `sheet['A4'] = '=SUM(A1:A3)'` tworzymy formułę wyliczaną automatycznie. W module `openpyxl.utils` można znaleźć między innymi dwie funkcje konwertujące liczby na litery i odwrotnie, co ułatwia iterowanie między komórkami.

In [ ]:
from openpyxl.utils import get_column_letter, column_index_from_string

print('Letters:')
print(",".join([get_column_letter(i) for i in range(1, 100)]))
print(column_index_from_string('A'))
print(column_index_from_string('AB'))
print(column_index_from_string('ABC'))

### Format _JSON_

Wczytywanie i zapisywanie formatu _JSON_ polega głównie na operowaniu na słownikach. Funkcja `loads` zwraca słownik na podstawie przekazanego łańcucha znaków w formacie _JSON_. Z kolei funkcja `dumps` zapisuje słownik w tym formacie. Można również użyć funkcji `load` i `dump`, która oczekuje strumienia do pliku zapisanego w formacie _JSON_ (otwarcie strumienia następuje w funkcji `open`).

In [ ]:
from json import loads, dumps

json_dict = loads('{ "file_name": "test", "is_zipped": false }')
print(f'File name: {json_dict["file_name"]}')
print(f'Is zipped: {json_dict["is_zipped"]}')
print(f'Dumped: {dumps(json_dict)}')

Warto w tym miejscu należy zwrócić uwagę na konwersję typów. Wartość typu `bool` w formacie _JSON_ została skonwertowana na odpowiednik w języku Python.

### Format _Parquet_

Format `Parquet` jest używany m. in. w _Apache Spark_ (_PySpark_) czy module `pandas` i jest jednym z podstawowych formatów w ekosystemie firmy _Apache_. Pod względem struktury jest to format binarny, kolumnowy z silnym typowaniem danych. W bibliotece standardowej nie ma obsługi tego formatu, jednak znajduje się wiele implementacji w `pip`. Przykłady utylizują bibliotekę `fastparquet`, a do tworzenia bibliotekę `pandas`.

In [ ]:
import pandas as pd
from tempfile import mktemp
from os import unlink
from fastparquet import ParquetFile

data = { 'A': list(range(1,5)), 'B': list(range(6,10)) }

print(f'Using data: {data}')

df = pd.DataFrame.from_dict(data)

temp_file = mktemp()

try:
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    print(f'Readed columns: {pf.columns}')
    print(f'File info: {pf.info}')
    df = pf.to_pandas()
    for value_a, value_b in df.itertuples(index=False):
        print(f'Values: {value_a}, {value_b}')

finally:
    unlink(temp_file)

Format `Parquet` jest szczególnie przydatny, gdy przetwarzamy duże zbiory danych i wynik należy zapisać, gdyż kolejny krok przetwarzania może zmodyfikować aktualnie przeprocesowane dane (_check point_).

## Baza danych _sqlite_ i podstawy `SqlAlchemy`

Mapowanie bazy danych na klasy jest bardzo przydatne w większych projektach. Głównie za sprawą podpowiadania składni oraz teoretycznej niezależności działania programu od warstwy przechowywania danych (baza danych). Biblioteka `SqlAlchemy` stanowi warstwę pośrednią miedzy fizycznymi operacjami na konkretnej bazie danych oraz logicznych operacji jak odczytywanie czy modyfikowanie danych. Co więcej, model danych to klasa w języku Python, co pozwala przy użyciu jednej funkcji założenie całej bazy danych z indeksami, tabelami i sekwencjami. Baza danych _sqlite_ jest wspierana w języku Python poprzez moduł `sqlite3` znajdujący się w bibliotece standardowej. Poniższy listing przedstawia prosty model danych wraz z przykładowym użyciem biblioteki `SqlAlchemy`.

In [ ]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system

temp_db = mktemp(suffix='.sqlite')

print(f'Using {temp_db}')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}

# w Windows dodaje dodatkowo /
engine = create_engine(conn_uri_handler[system()])

Base = declarative_base(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    id=Column(Integer, primary_key=True)
    title=Column('title', String(32))
    in_stock=Column('in_stock', Boolean)
    quantity=Column('quantity', Integer)


Base.metadata.create_all()

Session = sessionmaker(bind=engine)

@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}')
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) # dodanie parametru session do listy
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')

        return None

    return wrapper

@provide_session
def populatate(session):
    session.add(Product(id=1, title='Laptop Sony', in_stock=True, quantity=10))
    session.add(Product(id=2, title='Laptop Lenovo', in_stock=True, quantity=1))
    session.add(Product(id=3, title='Laptop Lenovo', in_stock=False, quantity=0))


@provide_session
def query(session):
    available_products = session.query(Product).filter(Product.in_stock == True).all()
    print('Available products')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')

    available_products = session.query(Product).filter(Product.in_stock == False).all()
    print('Not available products')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.quantity}')


populatate()
query()

W zależności od tego co zostanie przekazane do funkcji `create_engine`, `SqlAlchemy` utworzy takie połączenie do bazy danych i skonwertuje typy języka Python na odpowiadające im typy w konkretnej bazie danych. Użyta tabela jest raczej trywialna, `SqlAlchemy` daje znacznie więcej możliwości np. tworzenia relacji (jeden do jeden, jeden do wielu, wiele do wielu), ograniczeń (_constraint_), wywoływania funkcji wbudowanych oraz tworzenia własnych definicji funkcji i typów, które zostaną przetransformowane do konkretnego dialektu (składnia danego języka zapytań obsługiwanego w bazie danych np. `t-sql` - _Microsoft SQL Server, 'pl/sql' - Oracle). W powyższym przykładzie, w funkcji `create_session` tworzona jest sesja do bazy danych. Użyty mechanizm pozwala na automatyczny `commit` i zwalnianie połączenia i przekazywanie go do puli otwartych połączeń. Dekorator `provide_session` dodaje parametr `session` do dekorowanej funkcji. Powyższy kod nie jest przeznaczony dla wielu wątków.

## Serializacja i deserializacja z modułem `pickle`

Moduł `pickle` umożliwia zapisywanie na dysk istniejących w pamięci instancji typów, które po ponownym uruchomieniu programu mogą być odczytywane z dysku. Nie każdy typ jest obsługiwany, ale kolekcje takie jak lista, słownik czy nazwana tupla są obsługiwane. Rozszerzeniem modułu `pickle` są moduły `dill` i `cpickle`, które obsługują większą liczbę typów.

In [ ]:
from pickle import dump, load
from tempfile import mktemp
from os import unlink

temp_file = mktemp()

try:
    collection = ['A', 'B', 1, 2, True, False]
    print(f'Before: {collection}')
    with open(temp_file, 'wb') as f:
        dump(collection, f)
        f.flush()

    with open(temp_file, 'rb') as f:
        temp_col = load(f)
        print(f'After: {temp_col}')
finally:
    unlink(temp_file)


## Pamięć podręczna i moduł _functools_

Twórcy języka Python wbudowali w moduły standardowe dekoratory, które umożliwiają tworzenie pamięci podręcznej (_cache_).
Dostępnych jest parę metod, niemniej jednak schemat użycia jest bardzo podobny.

In [ ]:
from functools import lru_cache, cached_property


@lru_cache(maxsize=100)
def func_lru_cached(a: int, b: int):
    return a + b

glob_a, glob_b = 10,10

@cached_property
def sum_ab():
    return glob_a + glob_b

print(sum_ab)
print(func_lru_cached(glob_a, glob_b))

_lru_ oznacza _Least Recently Used_, co oznacza, że wywołania z parametrami rzadko używanymi będą usuwane z pamięci podręcznej w momencie kiedy tej pamięci podręcznej będzie brakować (parametr `maxsize`). Dekorator `cached_property` używany jest głównie do właściwości przechowywanych w klasach, które są przedmiotem kolejnego laboratorium.

## Zadania do wykonania

1. Porównaj zapisywanie i odczytywanie kolekcji (100, 10000, 100 000 elementów) za pomocą trzech technik: modułu `pickle`, `parquet` i `xlsx`.

### `pickle`

In [ ]:
from datetime import datetime
from tempfile import mktemp
from os import unlink
from pickle import dump, load

temp_file = mktemp()

amount = 100
collection = {}
for i in range(1, amount + 1):
    collection['A' + str(i)] = int(i)

pickle_start = datetime.now().timestamp()
try:
    with open(temp_file, 'wb') as f:
        dump(collection, f)
        f.flush()
    with open(temp_file, 'rb') as f:
        temp_col = load(f)
        # print(f'{temp_col}')
finally:
    unlink(temp_file)
pickle_end = datetime.now().timestamp()

print(round(pickle_end - pickle_start, 6))

### `parquete`

In [ ]:
from datetime import datetime
import pandas as pd
from tempfile import mktemp
from os import unlink
from fastparquet import ParquetFile
from openpyxl.utils import get_column_letter, column_index_from_string

temp_file = mktemp()

amount = 100
collection = {}
for i in range(1, amount + 1):
    collection['A' + str(i)] = [int(i)]

df = pd.DataFrame.from_dict(collection)

parquete_start = datetime.now().timestamp()
try:
    df.to_parquet(temp_file, compression='GZIP')
    pf = ParquetFile(temp_file)
    df = pf.to_pandas()
    for value_a in df.itertuples(index=False):
        temp_val = value_a
        # print(temp_val)
finally:
    unlink(temp_file)
parquet_end = datetime.now().timestamp()

print(parquet_end - parquete_start)

### `xlsx`

In [ ]:
from openpyxl import Workbook, load_workbook
from typing import Dict
from tempfile import mktemp
from os import unlink

file_name = mktemp(suffix='.xlsx')

amount = 100
collection = {}
for i in range(1, amount + 1):
    collection['A' + str(i)] = int(i)

def create(data: Dict[str, int], file_name, sheet_name):
    wb = Workbook()
    sheet = wb.create_sheet(sheet_name)
    for cell_key, cell_value in data.items():
        sheet[cell_key] = cell_value
    wb.save(file_name)

def describe(file_name, sheet_name):
    wb = load_workbook(file_name)
    for coll_obj in list(wb[sheet_name].columns)[0]:
        temp_val = coll_obj.value
        # print(temp_val)

try:
    xlsx_start = datetime.now().timestamp()
    create(collection, file_name, 'test')
    describe(file_name, 'test')
    xlsx_end = datetime.now().timestamp()

finally:
    unlink(file_name)

print(xlsx_end - xlsx_start)

2. Zbadaj przyspieszenie związane z zastosowaniem pamięci podręcznej na wybranych przykładzie funkcji rekurencyjnej (np. ciągu _Fibonacciego_).

Bez akceleracji dla 30 elementów

In [ ]:
from datetime import datetime

def fib_without_acce(n):
    if n < 2:
        return n
    return fib_without_acce(n-1) + fib_without_acce(n-2)

dtn = datetime.now().timestamp()
[fib_without_acce(n) for n in range(30)]
dte = datetime.now().timestamp()
print(dte - dtn)

Z akceleracją dla 1000 elementów

In [ ]:
import functools
from datetime import datetime

@functools.lru_cache(maxsize=None)
def fib_with_acce(n):
    if n < 2:
        return n
    return fib_with_acce(n-1) + fib_with_acce(n-2)

dtn = datetime.now().timestamp()
[fib_with_acce(n) for n in range(1000)]
dte = datetime.now().timestamp()
print(dte - dtn)

3. Napisz program tworzący bazę danych z interfejsem konsolowym. Wymagane są następujące operacje, dodanie wiersza, usunięcie wiersza, zmiana pola wiersza, wyświetlenie opcji. Menu można zorganizować jako odczytywanie parametrów zwróconych przez funkcję `input` lub z pomocą komend (łatwiejszy sposób).

In [ ]:
from sqlalchemy import *
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from contextlib import contextmanager
from tempfile import mktemp
from platform import system

temp_db = mktemp(suffix='.sqlite')

conn_uri_handler = {
    'Windows': f'sqlite:///{temp_db}',
    'Darwin': f'sqlite:////{temp_db}',
    'Linux': f'sqlite:////{temp_db}',
    'Java': f'sqlite:////{temp_db}'
}
# w Windows dodaje dodatkowo /
engine = create_engine(conn_uri_handler[system()])

Base = declarative_base(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    id=Column(Integer, primary_key=True)
    title=Column('title', String(32))
    in_stock=Column('in_stock', Boolean)
    quantity=Column('quantity', Integer)

Base.metadata.create_all()

Session = sessionmaker(bind=engine)
@contextmanager
def create_session():
    session = Session()
    try:
        yield session
        session.commit()
    except Exception:
        session.rollback()
        raise
    finally:
        session.close()

def provide_session(func):
    def wrapper(*args, **kwargs):
        try:
            with create_session() as session:
                args = (*args, session) if args else (session,) # dodanie parametru session do listy
                return func(*args, **kwargs)
        except Exception as e:
            print(f'Error found: {e}')
        return None
    return wrapper

@provide_session
def row_add(session):
    print('Product name:')
    title = input()
    print('Available amount')
    quantity = int(input())
    if quantity == 0:
        stock = False
    else:
        stock = True
    session.add(Product(title=title, in_stock=stock, quantity=quantity))
    print(f'Add: {title} in amount {quantity}')

@provide_session
def row_remove(session):
    print('Remove row with id:')
    search = input()
    session.query(Product).filter(Product.id == int(search)).delete()
    session.commit()

@provide_session
def row_change(session):
    print('Update row with id:')
    search = input()
    print('New product name: ')
    name = input()
    print('New amount: ')
    amount = input()
    session.query(Product).filter(Product.id == int(search)).update({
      Product.title: name, Product.quantity(): amount
    }, synchronize_session = False)

@provide_session
def query(session):
    available_products = session.query(Product).filter(Product.in_stock == True).all()
    print('Available products')
    for product in available_products:
        print(f'{product.id}\t{product.title}\t{product.in_stock}\t{product.quantity}')

    available_products = session.query(Product).filter(Product.in_stock == False).all()
    if len(available_products) > 0:
        print('Not available products')
        for product in available_products:
            print(f'{product.id}\t{product.title}\t{product.in_stock}\t{product.quantity}')


def show_options():
    print("\n1: Add new row\n2: Remove row\n3: Change row values\n4: Show database\n")
    mode_select = input()
    if mode_select == '1':
        try:
            row_add()
            show_options()
        except Exception as e:
            print(f'{e}')
    elif mode_select == '2':
        try:
            row_remove()
            show_options()
        except Exception as e:
            print(f'{e}')
    elif mode_select == '3':
        try:
            row_change()
            show_options()
        except Exception as e:
            print(f'{e}')
    elif mode_select == '4':
        query()
        show_options()
    elif mode_select == '0':
        exit()
    else:
        print('Wrong mode')
        show_options()

show_options()